In [1]:
import cv2
import numpy as np
import keying
import projection
import imp
import time

In [2]:
imp.reload(keying)

<module 'keying' from '/home/ozym4nd145/Coding/Project/Vision_Project/keying.py'>

In [3]:
def show_images(lst):
    while True:
        for i in lst:
            cv2.imshow(i[0],i[1])
        ret = cv2.waitKey(0)
        if ret==27:
            break
    cv2.destroyAllWindows()

In [4]:
def project_img(img,bg,mask,key_param,tola=16, tolb=50, low_thresh=0.05,
                high_thresh=0.25, sz=5, space=200,sat_mul_lo=5,sat_mul_hi=7):
    key_mask = keying.get_mask(img,key_param[0],tola, tolb, low_thresh, high_thresh, sz, space)
#     mod_img =  keying.process_img(img,key_param[1],sat_mul_lo,sat_mul_hi)
    mod_img = img
    bgra = keying.get_bgra(mod_img,key_mask)
    res = projection.project_to_mask(bgra,mask)
    trans_img = res[:,:,:3] ##(x,y,3)
    trans_mask = np.expand_dims((res[:,:,3]).astype(np.float32)/255,-1) ##(x,y)
    
#     r1 = np.multiply(trans_img,trans_mask)
    t_mask = np.tile(trans_mask,3)
    
    trans_img = trans_img.astype(np.float32)
    bg = bg.astype(np.float32)
    
    r1 = cv2.multiply(trans_img,t_mask)
    r2 = cv2.multiply(bg,1-t_mask)

    result = cv2.add(r1,r2)
    result = result.astype(np.uint8)
    return (result,trans_mask)

In [5]:
def get_key_param(img):
    key_region = keying.get_region(img)
    ycrcb = keying.get_params_ycrcb(img,key_region)
    hls = keying.get_params_hls(img,key_region)
    return (ycrcb,hls)

## IMG Test

In [6]:
img = cv2.imread("./image/twoperson.png")
bg = cv2.imread("./image/bg.jpg")
mask = cv2.imread("./image/mask.jpg")

key_param = get_key_param(img)
res,m = project_img(img,bg,mask,key_param)

show_images([("i",img),("bg",bg),("m",mask),("res",res)])

## Video Test

In [14]:
green_vid_path = "./video/fight_green.mp4"
# green_vid_path = "./video/suyash_oneplus.mp4"
env_vid_path = "./video/fight.avi"
env_mask_path = "./video/fight_mask.avi"
outp_vid = "./video/fight_result.mov"
outp_vid_l = "./video/fight_result_light.mov"

In [15]:
cap_green = cv2.VideoCapture(green_vid_path)
cap_vid = cv2.VideoCapture(env_vid_path)
cap_mask = cv2.VideoCapture(env_mask_path)

In [16]:
ret,img = cap_green.read()
# img = np.transpose(img,[1,0,2])
ret,bg = cap_vid.read()
ret,mask = cap_mask.read()

# img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
# bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
# mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))

In [10]:
key_param = get_key_param(img)

In [17]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap_vid.get(5)
vid = cv2.VideoWriter(outp_vid,fourcc, fps, (bg.shape[1],bg.shape[0]))
# livid = cv2.VideoWriter(outp_vid_l,fourcc, fps, (bg.shape[1],bg.shape[0]))

In [18]:
while True:
    start_t = time.time()
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    ret3,mask = cap_mask.read()
    
    if (not(ret1 and ret2 and ret3)):
        break
        
#     img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
#     bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
#     mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))
#     img = np.transpose(img,[1,0,2])    
    
    res,m = project_img(img,bg,mask,key_param)
#     r = np.expand_dims(10*cv2.blur(m*(1-m),(5,5)),-1)
#     light = bg*r
#     diffl = res*(1-r)
#     newi = (light+diffl).astype(np.uint8)
    
    vid.write(res)
#     livid.write(newi)
    
    end_time = time.time()
    print(end_time-start_t)
    
#     cv2.imshow("result",res)
# #     cv2.imshow("mask",m)
#     cv2.imshow("diff",r)
# #     cv2.imshow("light",light.astype(np.uint8))
# #     cv2.imshow("diffl",diffl.astype(np.uint8))
#     cv2.imshow("newl",newi)
#     ret = cv2.waitKey(20)
#     if ret==27:
#         break
cv2.destroyAllWindows()
vid.release()
livid.release()

0.23008060455322266
0.19672036170959473
0.1950681209564209
0.18555331230163574
0.24228906631469727
0.19295382499694824
0.20323967933654785
0.24498677253723145
0.18965411186218262
0.18894219398498535
0.2566807270050049
0.19444966316223145
0.2555694580078125
0.2124347686767578
0.19300460815429688
0.18101763725280762
0.25270533561706543
0.21937203407287598
0.20306825637817383
0.19678306579589844
0.18491744995117188
0.20277118682861328
0.18672871589660645
0.23718619346618652
0.1944742202758789
0.1875288486480713
0.18709158897399902
0.1912226676940918
0.18650555610656738
0.18052411079406738
0.2575051784515381
0.18829131126403809
0.1867048740386963
0.19736146926879883
0.2077796459197998
0.19980621337890625
0.18440628051757812
0.18396306037902832
0.2600977420806885
0.2783658504486084
0.2277686595916748
0.19220852851867676
0.2491614818572998
0.21601343154907227
0.1911153793334961
0.18938636779785156
0.1904456615447998
0.21512246131896973
0.19035696983337402
0.18272185325622559
0.19779968261718

KeyboardInterrupt: 

In [ ]:
def test():
    ret1,img = cap_green.read()
    ret2,bg = cap_vid.read()
    ret3,mask = cap_mask.read()
    
    if (not(ret1 and ret2 and ret3)):
        break
        
#     img = cv2.resize(img,(img.shape[1]//2,img.shape[0]//2))
#     bg = cv2.resize(bg,(bg.shape[1]//2,bg.shape[0]//2))
#     mask = cv2.resize(mask,(mask.shape[1]//2,mask.shape[0]//2))
#     img = np.transpose(img,[1,0,2])    
    
    res,m = project_img(img,bg,mask,key_param)
    r = np.expand_dims(10*cv2.blur(m*(1-m),(5,5)),-1)
    light = bg*r
    diffl = res*(1-r)
    newi = (light+diffl).astype(np.uint8)
    
    vid.write(res)
    livid.write(newi)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f project_img test()

In [ ]:
a = np.ones((2,3,1))

In [ ]:
a

In [ ]:
np.tile(a,3)